# Many-body simulations - $S(q)$ with 1-bead model

# Import packages

In [ ]:
import os
import numpy as np
from jinja2 import Template
import matplotlib.pyplot as plt
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 100%; }
    div#menubar-container     { width: 100%; }
    div#maintoolbar-container { width: 100%; }
</style>
"""))

plt.rcParams.update({'font.size':16,'legend.frameon':True,'figure.figsize':[12,8],'xtick.major.size':7,'ytick.major.size':7,'legend.labelspacing':1})

# Go and write down the working directory

In [ ]:
try:
    workdir
except NameError:
    workdir=%pwd
else:
    %cd -q $workdir

%cd $workdir

# Functions

In [ ]:
# Length of the cube side in Å corresponding to some protein concentration in g/l
def Length_in_gl(cp, N_of_proteins, Mw):
    return np.cbrt(N_of_proteins/(cp*(1/Mw)*6.022e23*1e-27)) 

# Create 1 bead model in pqr format 
def createpqr_1b(filename, sigma):
    beads = 1
    f = open(filename,'w')
    f.write(str(beads)+'\n')
    for i in range(beads):
        if i ==0:
            f.write('{0:6} {1:4} {2:4} {3:4} {4:5} {5:11.3f} {6:8.3f} {7:8.3f} {8:8.3f} {9:6.3f}\n'\
                .format('ATOM', i, 'BEAD', 'A', 0 , 0, 0, 0, 0, sigma/2))
    f.write('END'+'\n')        
    f.close()

# Create simulation input in yml format
def create_input(macro, micro, nstep, nskip, L, N, sigma, q, Dl, eps, p):
    with open('input.yml', 'w') as input_file:
        d = input_file.write(Input.render(macro = macro,
                                          micro = micro,
                                          nstep = nstep,
                                          nskip = nskip,
                                              L = L,
                                              N = N,
                                          sigma = sigma,
                                              q = q,
                                             Dl = Dl,
                                            eps = eps,
                                              p = p
                                          
                                         ))
        return d 

def kappa(conc, Z, rhob, rhos, sigma): # sigma in nm
    lB = 0.714 #nm
    Na = 6.02214076e23 # 1/mol
    rho = 4.068e15 #particles per ml
    phi = conc*rho*1e-21 * np.pi* sigma**3 / 6
    PHI = 1 / (1 - phi)
    k =  4 * np.pi * lB * ( PHI * conc*Z*rho*1e-21 + 2*rhob*Na*1e-27 +2*rhos*Na*1e-27 ) 
    return np.sqrt(k)

# Runs

In [ ]:
# Go and write down the working directory

try:
    workdir
except NameError:
    workdir=%pwd
else:
    %cd -q $workdir

%cd $workdir
####################

Input = Template("""
temperature: 298.15
geometry: {type: cuboid, length: [{{L}},{{L}},{{L}}]}
mcloop: {micro: {{micro}}, macro: {{macro}}}
random: {seed: hardware}
energy:
    - nonbonded:
        cutoff_g2g: {{1.2*sigma+8*Dl}} 
        default:
            - hardsphere:
                mixing: arithmetic 
            - custom:
                function: lB * q^2 * ( exp( sigma / (2 * Dl) ) / ( 1 + sigma / (2 * Dl) ) )^2 * ( exp(-r / Dl) / r ) - eps * ( sigma / r )^6         
                constants:
                    lB: 7.14
                    q: {{q}}
                    Dl: {{Dl}}
                    sigma: {{sigma}}
                    eps: {{eps}}  
atomlist:
    - BEAD:   { sigma: {{sigma}}}      
moleculelist:
    - 1_bead:
        structure: input.pqr
        insdir: [1,1,1]
        insoffset: [0,0,0]
        keepcharges: False
        keeppos: False
        rigid: True

insertmolecules:
    - 1_bead: {N: {{N}}, inactive: False}

moves:
    - moltransrot: {molecule: 1_bead, dir: [1,1,1], dprot: 1, dp: 100 }

analysis:
#    - sanity: {nstep: {{nstep}} }
#    - systemenergy: {file: energy.dat, nstep: {{nstep}}, nskip: {{nskip}}}
#    - molecule_density: {nstep: {{nstep}}}
    - savestate: {file: state.json}
    - savestate: {file: confout.pqr}
#    - xtcfile: {file: traj.xtc, nstep: {{nstep}}, nskip: {{nskip}} }
#    - molrdf: {file: rdf.dat, nstep: 1, dr: 1, name1: 1_bead, name2: 1_bead, dim: 3}
    - scatter: {file: debye.dat, nstep: 1, molecules: ["1_bead"], com: False, scheme: explicit , pmax: {{p}}}

""")
##############################################################################################################

####### Conditions ########
Mw = 148000               #
N = 1000                  #
###########################################
potential = 'HS_Yukawa_LJ'                              
c_range = [ 20, 50, 100, 150, 200 ]    # for 7 mM
#c_range = [ 10, 30, 50, 100, 150 ]    # for 57 mM
eps_range = [ 0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6]                     
I = [ 7 ]                                           
Z = [ 16, 18, 20, 22, 24, 26, 28, 30 ]                      
###########################################
micro = 1      #                                                                                     
macro = 10000  #
nstep = 1000   #
nskip = 0      #
################


### Runs ###
%mkdir runs
%cd runs
%mkdir $potential
%cd $potential


for i in I:
    %mkdir $i
    %cd -q $i
    for c in c_range:
        %mkdir $c
        %cd -q $c
        for z in Z:
            %mkdir $z
            %cd -q $z
            for eps in eps_range:
                %mkdir $eps
                %cd $eps
                
                if i ==7:
                    sigma = 96.4
                if i ==57:
                    sigma = 90

                L = Length_in_gl(c, N, Mw)
                Dl = 10/kappa(c, z, i, 0, sigma/10 )
                createpqr_1b('input.pqr', sigma )
                create_input(macro, micro, nstep, nskip, L, N, sigma, z, Dl, eps, 25)

                exists = os.path.isfile('state.json')
                if exists:
                    # Path for Faunus executable

                    !export OMP_NUM_THREADS=1;yason.py Input.yml | faunus -v2 --state state.json #> out    
                else:
                    # Path for Faunus executable

                    !export OMP_NUM_THREADS=1;yason.py Input.yml | faunus -v 2  #> out
                              
                %cd -q ../ 
            %cd -q ../
        %cd -q ../
    %cd -q ../ 

%cd -q ../../